In [1]:
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "portfolio_management.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from common.models import Brokers, Prices, Transactions, FX, Assets
from users.models import CustomUser
from utils import NAV_at_date, Irr, chart_dates, chart_labels, chart_colour, portfolio_at_date, calculate_security_nav, currency_format, calculate_portfolio_value
from datetime import date
from django.db.models import F, Sum

In [2]:
selected_brokers = broker_ids = [2]
effective_current_date = date.today()
currency_target = 'USD'
number_of_digits = 2

CustomUser.objects.get(id=2)

<CustomUser: Yaroslav>

In [3]:
from pyxirr import xirr


def Irr(date, currency, asset_id=None, broker_id_list=None, start_date=''):
    
    # Calculate portfolio value
    portfolio_value = calculate_portfolio_value(date, currency, asset_id, broker_id_list)

    if portfolio_value < 0:
        return 'N/R'
    
    cash_flows = []
    transaction_dates = []

    # Collect cash flows and transaction dates for the portfolio
    transactions = Transactions.objects.filter(date__lte=date, security_id=asset_id)
    
    # if asset_id:
    #     transactions = transactions.filter(security_id=asset_id)

    if broker_id_list:
        transactions = transactions.filter(broker_id__in=broker_id_list)

    if start_date:
        transactions = transactions.filter(date__gte=start_date)

    # print(f"utils.py. line 123. Transactions: {transactions}")

    # Calculate cash flows and transaction dates
    for transaction in transactions:
        print(f"utils.py. line 127. Transaction details: {transaction.quantity}")
        fx_rate = FX.get_rate(transaction.currency.upper(), currency, transaction.date)['FX']

        if transaction.type == 'Cash in' or transaction.type == 'Cash out':
            cash_flow = -transaction.cash_flow
        else:
            cash_flow = transaction.cash_flow or (-transaction.quantity * transaction.price + (transaction.commission or 0))
        
        cash_flows.append(round(cash_flow * fx_rate, 2))
        transaction_dates.append(transaction.date)

    # Calculate start portfolio value if provided
    if start_date:
        start_portfolio_value = calculate_portfolio_value(start_date, currency, asset_id, broker_id_list)
        cash_flows.insert(0, -start_portfolio_value)
        transaction_dates.insert(0, start_date)

    # Calculate portfolio value
    portfolio_value = calculate_portfolio_value(date, currency, asset_id, broker_id_list)

    # Append end portfolio value to cash flows and dates
    cash_flows.append(portfolio_value)
    transaction_dates.append(date)

    print(f"utils. line 146. Cash flows: {cash_flows}, Transaction dates: {transaction_dates}")

    try:
        return round(xirr(transaction_dates, cash_flows), 4)
    except:
        return 'N/A'

# def calculate_portfolio_value(date, currency=None, asset_id=None, broker_id_list=None):
#     portfolio_value = 0

#     if asset_id is None:
#         portfolio_value = NAV_at_date(broker_id_list, date, currency, [])['Total NAV']
#     else:
#         asset = Assets.objects.get(id=asset_id)
#         # print(f"utils.py. line 165. asset current price: {asset.current_price(date)}")
#         # print(f"utils.py. line 167. Asset data: {asset.current_price(date).price}, {asset.position(date, broker_id_list)}")
#         portfolio_value = round(asset.price_at_date(date, currency).price * asset.position(date, broker_id_list), 2)

#     return portfolio_value

security_id = 4
security = Assets.objects.get(id=security_id)
print('Price:', security.price_at_date(effective_current_date).price)
print('Value:', calculate_portfolio_value(effective_current_date, security.currency, security_id))

Irr(effective_current_date, security.currency, security_id)

Price: 165.00


TypeError: 'NoneType' object is not iterable

In [ ]:
security_id = 7
security = Assets.objects.get(id=security_id)
security.price_at_date(effective_current_date).price

quote = security.prices.filter(date__lte=effective_current_date).order_by('-date').first()
print(currency_target)
if currency_target is not None:
    quote.price = quote.price * FX.get_rate(security.currency, currency_target, effective_current_date)['FX']
quote

In [24]:
investor = CustomUser.objects.get(id=2)

from common.models import update_FX_from_Yahoo
from utils import update_fx_database
from django.db.models import Sum, F, FloatField, Case, When, Value
from constants import TOLERANCE

# print(investor.id)

# Get the specific investor
investor_instance = CustomUser.objects.get(id=investor.id)

t = Transactions.objects.filter(investor_id=investor.id).exclude(type='Cash in')
# t.delete()

a = investor.assets.all()
# for a in a:
#     print(a.name, a.position('2022-1-1'))

portfolio_open = Assets.objects.filter(
        investor=investor,
        transactions__date__lte=effective_current_date,
    ).prefetch_related(
        'transactions'
    ).annotate(
        total_quantity=Sum('transactions__quantity')
    ).annotate(
        total_quantity_adjusted=Case(
        When(total_quantity__lt=0, then=Value(0, output_field=FloatField())),
        default=F('total_quantity'),
        output_field=FloatField()
    )
).exclude(total_quantity_adjusted=0)

print(portfolio_open)

a = Assets.objects.filter(
        investor=investor,
).annotate(
    total_quantity=Sum('transactions__quantity')
).annotate(
        total_quantity_adj=Case(
            When(total_quantity__lt=TOLERANCE, then=Value(0, output_field=FloatField())),
            default=F('total_quantity'),
            output_field=FloatField()
        )
    )

portfolio_open = Assets.objects.filter(
    id=8,
    investor=investor,
).prefetch_related(
    'transactions'
).annotate(
    total_quantity=Sum('transactions__quantity')
).annotate(
    total_quantity_adj=Case(
        When(total_quantity__lt=TOLERANCE, then=Value(0, output_field=FloatField())),
        default=F('total_quantity'),
        output_field=FloatField()
    )
)

# If you need to get the specific object
asset = portfolio_open.first() 
asset
print(Assets.objects.get(id=8).position('2023-1-1'))
print(Assets.objects.get(id=8).transactions.aggregate(total=Sum('quantity'))['total'])

<QuerySet []>
0
-8.88178419700125E-16


In [ ]:
update_FX_from_Yahoo('PLN', 'USD', date(2011, 1, 10), 5)